In [53]:
#In this jupyter file, we write all the code and later it will be converted into seggreated as modular code in differenct files.

%pwd

'/Users/charan/Documents/LLM+RAG-medibot-project/GenAI-HealthChat'

In [54]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [55]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [56]:
extracted_data=load_pdf_file(data='Data/')

In [57]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [58]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [59]:
from langchain.embeddings import HuggingFaceEmbeddings

In [60]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [61]:
embeddings = download_hugging_face_embeddings()

/var/folders/p8/jh6yz_3j5r363sx08k8lclgw0000gn/T/ipykernel_45024/2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [78]:
from dotenv import load_dotenv
load_dotenv()

True

In [79]:
#Create the pinecode index in the pinecone website

#initializing the pinecode api key
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
OPENAI_ORG_KEY=os.environ.get('OPENAI_ORG_KEY')
HUGGINGFACEHUB_API_KEY = os.environ.get('HUGGINGFACEHUB_API_KEY')

In [80]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_ORG_KEY"] = OPENAI_ORG_KEY
os.environ["HUGGINGFACEHUB_API_KEY"] = HUGGINGFACEHUB_API_KEY

In [65]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name="medichat",
    embedding=embeddings, 
)

In [67]:
# Load Existing index : we load the data from pinecode and load it into a variable.

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name="medichat",
    embedding=embeddings
)

In [68]:
docsearch

In [69]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='1e13151c-fcd3-46bf-9307-29167af4fbc6', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='e14c5934-ea2c-4405-ab9e-2a5713bfd843', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 

In [ ]:
#User will give the quuery, this query will go to the knowledge base(vector db).
#This knowledge base gives the relevant rank results. Then, LLM will provess these rank results+user query to give the ai response.

In [93]:
#from langchain_openai import OpenAI
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4")
#llm = OpenAI(temperature=0.4, max_tokens=500)

# from langchain_huggingface import HuggingFaceEndpoint


# llm = HuggingFaceEndpoint(
#     repo_id="mistralai/Mistral-Large-2",
#     temperature=0.5,
#     max_new_tokens=128
# )

#Because we are using ollama, it should run in the background.

from langchain_community.llms import Ollama
llm = Ollama(model="mistral")

/var/folders/p8/jh6yz_3j5r363sx08k8lclgw0000gn/T/ipykernel_45024/532934093.py:25: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")


In [94]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [95]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [97]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

 Acromegaly is a rare disorder characterized by excessive growth in bones and soft tissues due to the abnormal release of a hormone from the pituitary gland. If the onset occurs after bone growth stops, it's called acromegaly. Gigantism refers to unusual height that results from this condition when it happens during growth periods. Acromegaly affects approximately 50 people per million and symptoms often appear in middle age due to their gradual development.
